In [1]:
import pandas as pd 

In [6]:
#loading dataframe
df = pd.read_csv('sentiment.csv')
df

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0000,yes,1.0000,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0000,yes,1.0000,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0000,yes,1.0000,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0000,yes,1.0000,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13866,13867,No candidate mentioned,1.0000,yes,1.0000,Negative,0.7991,Abortion,0.6014,No candidate mentioned,...,yes,7,Negative,Abortion\nWomen's Issues (not abortion though),RT @cappy_yarbrough: Love to see men who will ...,NaN,2015-08-07 09:29:43 -0700,629690895479250944,Como,NaN
13867,13868,Mike Huckabee,0.9611,yes,1.0000,Positive,0.7302,None of the above,0.9229,Mike Huckabee,...,yes,1,NaN,NaN,RT @georgehenryw: Who thought Huckabee exceede...,NaN,2015-08-07 09:25:02 -0700,629689719056568320,USA,NaN
13868,13869,Ted Cruz,1.0000,yes,1.0000,Positive,0.8051,None of the above,0.9647,Ted Cruz,...,yes,67,Positive\nNeutral,NaN,"RT @Lrihendry: #TedCruz As President, I will a...",NaN,2015-08-07 07:19:18 -0700,629658075784282112,NaN,NaN
13869,13870,Donald Trump,1.0000,yes,1.0000,Negative,1.0000,Women's Issues (not abortion though),0.9202,Donald Trump,...,yes,149,NaN,Women's Issues (not abortion though),RT @JRehling: #GOPDebate Donald Trump says tha...,NaN,2015-08-07 09:54:04 -0700,629697023663546368,NaN,NaN


In [8]:
#taking the imortant feature 
data = df[['text','sentiment']]
data

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive
...,...,...
13866,RT @cappy_yarbrough: Love to see men who will ...,Negative
13867,RT @georgehenryw: Who thought Huckabee exceede...,Positive
13868,"RT @Lrihendry: #TedCruz As President, I will a...",Positive
13869,RT @JRehling: #GOPDebate Donald Trump says tha...,Negative


In [46]:
#import important libs 
import numpy as np 
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential 
from keras.layers import Dense ,LSTM ,SpatialDropout1D ,Embedding
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re 


In [16]:
data = data[data.sentiment != 'neutral']

In [17]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')
    
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')

4472
16986


In [39]:
t = tokenizer.fit_on_texts(data['text'].values)
t

In [49]:
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

In [50]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))

In [55]:
model.predict(X).shape

336/336 [==============================] - 0s 533us/step


(10729, 28, 128)

In [33]:
X = pad_sequences(X,maxlen = 40)
X.shape

(10729, 40)

In [56]:
X

array([[   0,    0,    0, ..., 1324, 1409,  743],
       [   0,    0,    0, ...,  233,  724,   17],
       [   0,    0,    0, ...,  207,  371,  670],
       ...,
       [   0,    0,    0, ...,   72,   65,    3],
       [   0,    0,    0, ..., 1022, 1423,   74],
       [   0,    0,    0, ...,  197,    3,  723]])

In [59]:
model = Sequential()
model.add(Embedding(2000, 200,input_length = X.shape[1]))
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 28, 200)           400000    
                                                                 
 lstm_1 (LSTM)               (None, 200)               320800    
                                                                 
 dense_1 (Dense)             (None, 2)                 402       
                                                                 
Total params: 721,202
Trainable params: 721,202
Non-trainable params: 0
_________________________________________________________________
None


In [61]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)


In [62]:
batch_size = 64
model.fit(X_train, Y_train, epochs = 20, batch_size=batch_size, verbose = 2)

Epoch 1/20
113/113 - 14s - loss: 0.4367 - accuracy: 0.8114 - 14s/epoch - 126ms/step
Epoch 2/20
113/113 - 12s - loss: 0.3087 - accuracy: 0.8717 - 12s/epoch - 103ms/step
Epoch 3/20
113/113 - 13s - loss: 0.2611 - accuracy: 0.8898 - 13s/epoch - 113ms/step
Epoch 4/20
113/113 - 14s - loss: 0.2272 - accuracy: 0.9055 - 14s/epoch - 120ms/step
Epoch 5/20
113/113 - 14s - loss: 0.1963 - accuracy: 0.9192 - 14s/epoch - 120ms/step
Epoch 6/20
113/113 - 14s - loss: 0.1689 - accuracy: 0.9367 - 14s/epoch - 120ms/step
Epoch 7/20
113/113 - 13s - loss: 0.1510 - accuracy: 0.9403 - 13s/epoch - 117ms/step
Epoch 8/20
113/113 - 13s - loss: 0.1280 - accuracy: 0.9485 - 13s/epoch - 116ms/step
Epoch 9/20
113/113 - 13s - loss: 0.1200 - accuracy: 0.9495 - 13s/epoch - 118ms/step
Epoch 10/20
113/113 - 14s - loss: 0.1049 - accuracy: 0.9563 - 14s/epoch - 124ms/step
Epoch 11/20
113/113 - 14s - loss: 0.0976 - accuracy: 0.9598 - 14s/epoch - 122ms/step
Epoch 12/20
113/113 - 13s - loss: 0.0887 - accuracy: 0.9636 - 13s/epoch - 

In [69]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

56/56 - 1s - loss: 1.0458 - accuracy: 0.8229 - 1s/epoch - 19ms/step
score: 1.05
acc: 0.82


In [71]:
twt = ['donald trump is great man with ']
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(twt)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)
sentiment = model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")
print(sentiment)

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0  78  15   5 146 224  45]]
1/1 - 0s - 17ms/epoch - 17ms/step
positive
[0.00640966 0.99359035]
